# Data Collector - Lecture Notes

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Stanfod CS231n (Transcript, Lecture Note)

Transform leture note website to readable format (URL to MD)

In [ ]:
!pip install markdownify

In [ ]:
!pip install requests

In [ ]:
%cd /content/drive/MyDrive/Dash-7b/Dash-7b/sum

In [ ]:
!pip install youtube-transcript-api

In [ ]:
import pandas as pd
import urllib.request
import markdownify
from markdownify import markdownify
import requests
from bs4 import BeautifulSoup
import re
from youtube_transcript_api import YouTubeTranscriptApi

def convert_to_markdown(url):
    fp = urllib.request.urlopen(url)
    mybytes = fp.read()
    mystr = mybytes.decode("utf8")
    fp.close()

    options = {
        "heading_style": "ATX",
        "bullets": "*+-",
        "code_language": "python",
        "wrap": True,
        "wrap_width": 80,
    }

    markdown_text = markdownify(mystr, **options)
    return markdown_text

def get_youtube_transcript(video_id, language='en', time_range=None):
    transcript_data = YouTubeTranscriptApi.get_transcript(video_id, languages=[language])

    def in_time_range(start, end, time_range):
        if time_range is None:
            return True
        return time_range[0] <= start and time_range[1] >= end

    transcript_lines = []
    for line in transcript_data:
        start = line['start']
        end = start + line['duration']

        if in_time_range(start, end, time_range):
            transcript_lines.append(line['text'])

    transcript = ' '.join(transcript_lines)
    return transcript

df = pd.read_csv("urls.csv")
transcripts = []
markdowns = []

for _, row in df.iterrows():
    video_id = row['video_url']
    note_url = row['note_url']
    time_range_str = row['time_range']

    if pd.isna(time_range_str):
        time_range = None
    else:
        start, end = map(int, time_range_str.split(','))
        time_range = (start, end)

    transcript = get_youtube_transcript(video_id, time_range=time_range)
    markdown = convert_to_markdown(note_url)

    transcripts.append(transcript)
    markdowns.append(markdown)

df['transcript'] = transcripts
df['markdown'] = markdowns

df.to_csv("sumdata.csv", index=False)